In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import pandas as pd

import sys
sys.path.append('..')

from helpers import utils, pipelines, models

from sklearn.model_selection import train_test_split

import multiprocessing

from data_augmentaion.data_augmentator import DataAugmentor 

import json

import time

pd.set_option('display.max_columns', None)

### Define research parameters

In [3]:
regression_task = False
continuous_features = ['age', 'fnlwgt', 'hours-per-week']
# metric = 'f1'
metrics = ['f1', 'accuracy', 'precision', 'recall', 'roc_auc']
test_size_proportion = 0.2
augment_sample = 0.5
sample_frac = 0.05

search_pipelines = pipelines.get_classification_pipelines()
search_parameters = models.parameters

settings = [
    {'method': 'random'},
    {'method': 'smote'},
    {'method': 'cf_random'},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 0.2}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 1}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 7, 'diversity_weight': 0.2, 'sparsity_weight': 0.2}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 1}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 0.2}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 1}},
#     {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 0.2}},
#     {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 1}},
]

### Read and preprocess data

In [4]:
raw_data = pd.read_csv("../datasets/adult.csv")
df = utils.preprocess_adult(raw_data)
target = 'income'

#### split data

In [5]:
# all data
X = df.drop(target, axis=1)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_proportion, random_state=42)

# sampled data
df_sample = df.sample(frac=sample_frac, random_state=42)
X_sample = df_sample.drop(target, axis=1)
y_sample = df_sample[target]
X_sample_train, X_sample_test, y_sample_train, y_sample_test = train_test_split(X_sample, y_sample, test_size=test_size_proportion, random_state=42)

In [6]:
display(y_train.value_counts())
display(y_sample_train.value_counts())
print(f'y_train classes gap: {y_sample_train.value_counts().max()-y_sample_train.value_counts().min()}')

0    27131
1     9009
Name: income, dtype: int64

0    1344
1     463
Name: income, dtype: int64

y_train classes gap: 881


In [7]:
full_results_df = pd.DataFrame()

### Whole dataset scores

In [8]:
print(f'whole adult dataset scores:')
_, whole_scores = utils.fit_and_evaluate(X_train, y_train, X_test, y_test,
                    search_estimators=search_pipelines, search_params=search_parameters, scoring=metrics)
whole_dataset_result_df = pd.DataFrame.from_dict(whole_scores, orient='index')
whole_dataset_result_df.columns = pd.MultiIndex.from_product([['whole']] + [whole_dataset_result_df.columns])
display(whole_dataset_result_df)

whole adult dataset scores:


whole                                        
           f1  accuracy precision    recall   roc_auc
lg   0.611430  0.828224  0.676717  0.558596  0.883247
rf   0.557895  0.823354  0.777262  0.459188  0.877579
xgb  0.633659  0.833758  0.680795  0.592339  0.890927

### Sampled adult dataset scores

In [9]:
print(f'{sample_frac*100}% sampled adult dataset scores:')
_, sampled_scores = utils.fit_and_evaluate(X_sample_train, y_sample_train, X_sample_test, y_sample_test,
                                           search_estimators=search_pipelines, search_params=search_parameters, scoring=metrics)
sampled_dataset_result_df = pd.DataFrame.from_dict(sampled_scores, orient='index')
sampled_dataset_result_df.columns = pd.MultiIndex.from_product([['sample']] + [sampled_dataset_result_df.columns])
sampled_dataset_result_df

5.0% sampled adult dataset scores:


c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserW

sample                                        
           f1  accuracy precision    recall   roc_auc
lg   0.604396  0.845133  0.662500  0.572917  0.863150
rf   0.375000  0.823009  0.807692  0.250000  0.851826
xgb  0.558659  0.825221  0.602410  0.520833  0.848124

In [10]:
full_results_df = pd.concat([full_results_df, whole_dataset_result_df, sampled_dataset_result_df], axis=1)

### running experiments

In [11]:
best_estimators = {}
best_scores = {}
total_time = time.time()
for i, s in enumerate(settings):
    start = time.time()
    print(f'{i+1} / {len(settings)}, {s}', end=' ')
    augmentor = DataAugmentor(X_sample_train, y_sample_train, X_sample_test, y_sample_test,
                             method=s['method'], regression=regression_task,
                             continuous_feats=continuous_features,
                             kw_args=s.get('kw_args', {})
                             )

    X_train_augmented_balanced, y_train_augmented_balanced = augmentor.augment(balance=True)
    best_estimators[f'{i}_balanced'], best_scores[f'{i}_balanced'] = \
    utils.fit_and_evaluate(X_train_augmented_balanced, y_train_augmented_balanced, X_sample_test, y_sample_test,
                            search_estimators=search_pipelines, search_params=search_parameters, scoring=metrics)
    result_df_balanced = pd.DataFrame.from_dict(best_scores[f'{i}_balanced'], orient='index')
    result_df_balanced.columns = pd.MultiIndex.from_product([[f'{json.dumps((list(s.values())))} balanced']] + [result_df_balanced.columns])


    X_train_augmented, y_train_augmented = augmentor.augment(balance=False, size=augment_sample)
    best_estimators[f'{i}'], best_scores[f'{i}'] = \
        utils.fit_and_evaluate(X_train_augmented, y_train_augmented, X_sample_test, y_sample_test,
                               search_estimators=search_pipelines, search_params=search_parameters, scoring=metrics)
    result_df = pd.DataFrame.from_dict(best_scores[f'{i}'], orient='index')
    result_df.columns = pd.MultiIndex.from_product([[f'{json.dumps((list(s.values())))}']] + [result_df.columns])

    full_results_df = pd.concat([full_results_df, result_df_balanced, result_df], axis=1)
    print(f'{time.time() - start} seconds for settings {i}')

print(f'\nTotal time: {time.time() - total_time}')

1 / 4, {'method': 'random'} 

c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserW

18.238768577575684 seconds for settings 0
2 / 4, {'method': 'smote'} 

c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserW

18.310089588165283 seconds for settings 1
3 / 4, {'method': 'cf_random'} 

...

850/903


100%|██████████| 1/1 [00:00<00:00,  2.48it/s]


900/903


100%|██████████| 1/1 [00:00<00:00,  2.66it/s]
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\danie\anaconda3\lib\site-packages

959.4883267879486 seconds for settings 3

Total time: 2118.5575625896454


### Results

In [16]:
full_results_df

whole                                            sample            \
           f1  accuracy precision    recall   roc_auc        f1  accuracy   
lg   0.611430  0.828224  0.676717  0.558596  0.883247  0.604396  0.845133   
rf   0.557895  0.823354  0.777262  0.459188  0.877579  0.375000  0.823009   
xgb  0.633659  0.833758  0.680795  0.592339  0.890927  0.558659  0.825221   

                                  ["random"] balanced                      \
    precision    recall   roc_auc                  f1  accuracy precision   
lg   0.662500  0.572917  0.863150            0.603175  0.778761  0.487179   
rf   0.807692  0.250000  0.851826            0.578571  0.738938  0.440217   
xgb  0.602410  0.520833  0.848124            0.595918  0.780973  0.489933   

                        ["random"]                                         \
       recall   roc_auc         f1  accuracy precision   recall   roc_auc   
lg   0.791667  0.863003   0.616216  0.849558  0.662651  0.59375  0.860838   
rf   0.854167  0.855425   0.452055  0.823009  0.789474  0.34375  0.856156   
xgb  0.729167  0.846179   0.518919  0.803097  0.539326  0.50000  0.828959   

    ["smote"] balanced                                         ["smote"]  \
                    f1  accuracy precision    recall   roc_auc        f1   
lg            0.592920  0.792035  0.507937  0.718750  0.853377  0.592920   
rf            0.558824  0.734513  0.436047  0.812500  0.845140  0.558824   
xgb           0.590909  0.805310  0.534483  0.677083  0.845330  0.590909   

                                            ["cf_random"] balanced            \
     accuracy precision    recall   roc_auc                     f1  accuracy   
lg   0.792035  0.507937  0.718750  0.853377               0.587156  0.800885   
rf   0.734513  0.436047  0.812500  0.845140               0.582734  0.743363   
xgb  0.805310  0.534483  0.677083  0.845330               0.589286  0.796460   

                                  ["cf_random"]                                \
    precision    recall   roc_auc            f1  accuracy precision    recall   
lg   0.524590  0.666667  0.833743      0.591928  0.798673  0.520000  0.687500   
rf   0.490066  0.843750  0.851914      0.577465  0.743363  0.444444  0.854167   
xgb  0.503937  0.687500  0.836640      0.562771  0.792035  0.507812  0.677083   

               \
      roc_auc   
lg   0.835820   
rf   0.853435   
xgb  0.841643   

    ["cf_genetic", {"proximity_weight": 7, "diversity_weight": 0.2, "sparsity_weight": 0.2, "total_CFs": 1, "desired_class": 1}] balanced  \
                                                                                                                                       f1   
lg                                            0.608696                                                                                      
rf                                            0.593407                                                                                      
xgb                                           0.595556                                                                                      

                                             \
     accuracy precision    recall   roc_auc   
lg   0.800885  0.529851  0.739583  0.852031   
rf   0.754425  0.457627  0.843750  0.856815   
xgb  0.789823  0.503876  0.697917  0.843633   

    ["cf_genetic", {"proximity_weight": 7, "diversity_weight": 0.2, "sparsity_weight": 0.2, "total_CFs": 1, "desired_class": 1}]  \
                                                                                                                              f1   
lg                                            0.620087                                                                             
rf                                            0.581818                                                                             
xgb                                           0.591304                                                    

In [13]:
print('best methods')
best_methods = pd.DataFrame()
for s in metrics:
    models_scores = full_results_df.xs(s, axis='columns', level=1)
    max_models = models_scores.idxmax(axis='columns')
    max_models.name = s
    max_models['overall'] = models_scores.max(axis=0).idxmax(axis=0)
    best_methods = pd.concat([best_methods, max_models], axis=1)
best_methods

best methods


,f1,accuracy,precision,recall,roc_auc
lg,"[""cf_genetic"", {""proximity_weight"": 7, ""divers...","[""random""]",whole,"[""random""] balanced",whole
rf,"[""cf_genetic"", {""proximity_weight"": 7, ""divers...",whole,sample,"[""random""] balanced",whole
xgb,whole,whole,whole,"[""random""] balanced",whole
overall,whole,"[""random""]",sample,"[""random""] balanced",whole


In [14]:
full_results_df.to_csv(rf'../log/experiment_classification_adult_testsize{test_size_proportion}_augmentsample{augment_sample}.csv')